# Capstone week 3 part 3

Third part of the <b> Segmenting and Clustering Neighborhoods in Toronto </b> -assignment.

<b> Install and import required dependencies.</b>

In [3]:
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import requests
import numpy as np
from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors

<b> Load the data from the previous assignments.</b>

In [4]:
# Load the data
toronto_neighbourhoods = pd.read_csv('toronto_geospatial_postal_codes.csv')
# Inspect data
toronto_neighbourhoods

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


<b> Visualize the map of Toronto. </b>

In [5]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto is 43.6534817, -79.3839347.


In [6]:
# Generate at new folium map from latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

map_toronto

<b> Setup FourSquare credentials.</b>

In [7]:
CLIENT_ID = '0UIQC4X0V003NCE4WXXXBIJ3SP5VK14OY5KJ2KXPAOUNPM42'
CLIENT_SECRET = 'WKD5ZUULSGQGYJZG0AJKHLVZ5G3BEJWZTHK4UOB1DPLRJP2R'
VERSION = '20180604'
LIMIT = 100

<b> Explore Toronto neighbourhoods and venues.</b>

Re-use the venue query function from the course for acquiring venues for a neighbourhood given lat, lon. Disable print to avoid clutter.

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):

    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Apply getNearbyVenues for the whole dataframe.

In [11]:
toronto_venues = getNearbyVenues(
    names = toronto_neighbourhoods['Neighborhood'], 
    latitudes = toronto_neighbourhoods['Latitude'], 
    longitudes = toronto_neighbourhoods['Longitude'])

<b>Inspect the acquired data.</b>

In [12]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,649 Variety,43.754513,-79.331942,Convenience Store
3,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


In [13]:
toronto_venues.tail()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1987,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Subway,43.631659,-79.519001,Sandwich Place
1988,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Jim & Maria's No Frills,43.631152,-79.518617,Grocery Store
1989,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Islington Florist & Nursery,43.630156,-79.518718,Flower Shop
1990,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Koala Tan Tanning Salon & Sunless Spa,43.631370,-79.519006,Tanning Salon
1991,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Burrito Boyz,43.626657,-79.526349,Burrito Place


In [14]:
toronto_venues.shape

(1992, 7)

In [15]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Wilson Heights, Downsview North",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
...,...,...,...,...,...,...
"Willowdale, Newtonbrook",2,2,2,2,2,2
Woburn,4,4,4,4,4,4
Woodbine Heights,5,5,5,5,5,5


<b> Analyze venue data.</b>

In [16]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group the data by neighbourhood and take mean of the occurrance for future analyzing of the most common type of venues in the area.

In [17]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Generate new dataframe with the most common venues data per neighbourhoods.

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Clothing Store,Skating Rink,Latin American Restaurant,Breakfast Spot,Yoga Studio,Metro Station,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant
1,"Alderwood, Long Branch",Pizza Place,Gym,Skating Rink,Pharmacy,Pool,Pub,Sandwich Place,Coffee Shop,Vegetarian / Vegan Restaurant,Turkish Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Pharmacy,Sandwich Place,Fried Chicken Joint,Sushi Restaurant,Frozen Yogurt Shop,Deli / Bodega,Intersection,Gas Station
3,Bayview Village,Japanese Restaurant,Café,Bank,Chinese Restaurant,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Yoga Studio
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Restaurant,Sandwich Place,Pizza Place,Indian Restaurant,Pharmacy,Cupcake Shop,Pub,Café,Butcher


<b><h3>Cluster neighbourhoods.</h3></b>

Use k-means to cluster the neighbourhoods into five clusters

In [22]:
# Setup number of clusters
kclusters = 4

# Create dataframe for clusters and drop the Neighborhood column
toronto_clusters = toronto_grouped.drop('Neighborhood', 1)

# Cluster the data with k means
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clusters)

In [23]:
# Add cluster labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = toronto_neighbourhoods.copy
# Merge neighbourhoods and venues to get combined details for viewing
toronto_merged = pd.merge(toronto_neighbourhoods, neighborhoods_venues_sorted, how='right', on='Neighborhood')
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1S,Scarborough,Agincourt,43.794200,-79.262029,1,Lounge,Clothing Store,Skating Rink,Latin American Restaurant,Breakfast Spot,Yoga Studio,Metro Station,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant
1,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484,1,Pizza Place,Gym,Skating Rink,Pharmacy,Pool,Pub,Sandwich Place,Coffee Shop,Vegetarian / Vegan Restaurant,Turkish Restaurant
2,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,1,Bank,Coffee Shop,Pharmacy,Sandwich Place,Fried Chicken Joint,Sushi Restaurant,Frozen Yogurt Shop,Deli / Bodega,Intersection,Gas Station
3,M2K,North York,Bayview Village,43.786947,-79.385975,1,Japanese Restaurant,Café,Bank,Chinese Restaurant,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Yoga Studio
4,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,1,Coffee Shop,Restaurant,Sandwich Place,Pizza Place,Indian Restaurant,Pharmacy,Cupcake Shop,Pub,Café,Butcher


<b>Visually inspect the cluster on map.</b>

In [24]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<b><h3>Investigate clustered neighbourhoods.</h3></b>

In [25]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512,0,Park,Women's Store,Pool,Men's Store,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station,Medical Center
27,M3K,North York,DownsviewEast CFB Toronto,43.737473,-79.464763,0,Park,Airport,Yoga Studio,Men's Store,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station,Medical Center
49,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724,0,Park,Sandwich Place,Yoga Studio,Modern European Restaurant,Mac & Cheese Joint,Malay Restaurant,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant
69,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,0,Park,Playground,Trail,Yoga Studio,Men's Store,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station,Medical Center
83,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106,0,Park,Intersection,Convenience Store,Metro Station,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Yoga Studio
92,M9N,York,Weston,43.706876,-79.518188,0,Park,Convenience Store,Yoga Studio,Men's Store,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station,Mediterranean Restaurant
96,M2M,North York,"Willowdale, Newtonbrook",43.789053,-79.408493,0,Park,Yoga Studio,Molecular Gastronomy Restaurant,Mac & Cheese Joint,Malay Restaurant,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store
99,M2P,North York,York Mills West,43.752758,-79.400049,0,Park,Convenience Store,Yoga Studio,Men's Store,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station,Mediterranean Restaurant


The neighbourhoods in this cluster seem to be the areas with different activities like gyms, hockey arenas, dance and yoga studios and playgrounds.

In [26]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1S,Scarborough,Agincourt,43.794200,-79.262029,1,Lounge,Clothing Store,Skating Rink,Latin American Restaurant,Breakfast Spot,Yoga Studio,Metro Station,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant
1,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484,1,Pizza Place,Gym,Skating Rink,Pharmacy,Pool,Pub,Sandwich Place,Coffee Shop,Vegetarian / Vegan Restaurant,Turkish Restaurant
2,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,1,Bank,Coffee Shop,Pharmacy,Sandwich Place,Fried Chicken Joint,Sushi Restaurant,Frozen Yogurt Shop,Deli / Bodega,Intersection,Gas Station
3,M2K,North York,Bayview Village,43.786947,-79.385975,1,Japanese Restaurant,Café,Bank,Chinese Restaurant,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Yoga Studio
4,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,1,Coffee Shop,Restaurant,Sandwich Place,Pizza Place,Indian Restaurant,Pharmacy,Cupcake Shop,Pub,Café,Butcher
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,M2N,North York,Willowdale,43.770120,-79.408493,1,Coffee Shop,Pizza Place,Ramen Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Grocery Store,Ice Cream Shop,Electronics Store,Sandwich Place,Korean Restaurant
95,M2R,North York,Willowdale,43.782736,-79.442259,1,Coffee Shop,Pizza Place,Ramen Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Grocery Store,Ice Cream Shop,Electronics Store,Sandwich Place,Korean Restaurant
97,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Coffee Shop,Korean BBQ Restaurant,Mexican Restaurant,Luggage Store,Malay Restaurant,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store
98,M4C,East York,Woodbine Heights,43.695344,-79.318389,1,Beer Store,Park,Spa,Athletics & Sports,Curling Ice,Yoga Studio,Metro Station,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant


This cluster seems to contain the greener parts of the city with large number of parks and other amenities

In [27]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
71,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,2,Bar,Yoga Studio,Lounge,Mac & Cheese Joint,Malay Restaurant,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store


These neighbourhoods seem to be more downtown style areas with large concentration of restaurants and coffee shops.

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,M9M,North York,"Humberlea, Emery",43.724766,-79.532242,3,Baseball Field,Yoga Studio,Lounge,Mac & Cheese Joint,Malay Restaurant,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store
62,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,3,Baseball Field,Yoga Studio,Lounge,Mac & Cheese Joint,Malay Restaurant,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store


This neighbourhood seems to stand out from the other neighbourhood with its concentration of Baseball Field and yoga studios.